In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("amazonETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy/mm/dd")

# Show DataFrame
user_data_df.show(5)

In [ ]:
number_of_rows = user_data_df.select("customer_id").count()
print(f"The number of rows in the above dataset is {number_of_rows}")

In [ ]:
# Drop NaN Values
user_data_df = user_data_df.dropna()

In [ ]:
# Create dataframe to match schema

#   review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE
# );

review_id_df = user_data_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_df.show(5)

In [ ]:
# Changing review_date to DateType
from pyspark.sql.types import DateType
review_id_df = review_id_df.withColumn("review_date", review_id_df["review_date"].cast(DateType()))

In [ ]:
# Confirm datatypes
review_id_df.printSchema()

In [ ]:
#   products (
#   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#   product_title TEXT
# );

product_id_df = user_data_df.select(["product_id","product_title"]).distinct()
product_id_df.show(5)

In [ ]:
product_id_df.printSchema()

In [ ]:
# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );

customer_id_df = user_data_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customer_id_df.show(5)

In [ ]:
customer_id_df.printSchema()

In [ ]:
# CREATE TABLE vine_table (
#   review_id TEXT PRIMARY KEY,
#   star_rating INTEGER,
#   helpful_votes INTEGER,
#   total_votes INTEGER,
#   vine TEXT
# );

vine_df = user_data_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show(5)

In [ ]:
# Convert star_rating from string type to integer
from pyspark.sql.types import IntegerType
vine_df = vine_df.withColumn("star_rating", vine_df["star_rating"].cast(IntegerType()))
vine_df.show(5)

In [ ]:
vine_df.printSchema()

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://amazondb.cu4hemf5gle4.us-east-2.rds.amazonaws.com:5432/watches_reviewDB"
config = {"user":"postgres", 
          "password": "postgres1234", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
product_id_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customer_id_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)